In [1]:
import numpy as np
import csv
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import datasets, linear_model
import pickle
import io

In [90]:
xx = pd.read_pickle("X_train.p")

In [79]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
profiles_data = pd.read_csv('profiles.csv')

train_set = pd.read_csv("train.csv", index_col=[0,1])
train_set_unstacked = train_set.unstack(level=-1)
train_set_unstacked.fillna(0, inplace=True)

In [5]:
print "number of users missing age: %s" % sum(profiles_data['age'].isnull())
print "number of users missing sex: %s" % sum(profiles_data['sex'].isnull())
print "number of users missing country: %s" % sum(profiles_data['country'].isnull())

number of users missing age: 44842
number of users missing sex: 19535
number of users missing country: 0


In [3]:
df = pd.concat([profiles_data, pd.get_dummies(profiles_data['sex'], dummy_na=True)], axis=1)
df.rename(columns = lambda x: str(x), inplace=True)
df.rename(columns={'nan': 'sex-nan'}, inplace=True)
df = pd.concat([df, pd.get_dummies(df['country'], dummy_na=True)], axis=1)
df.rename(columns = lambda x: str(x), inplace=True)
df.rename(columns={'nan': 'country-nan'}, inplace=True)
del df['sex']
del df['country']
df['age-nan'] = np.isnan(df['age']) * 1
df['age-negative'] = (df['age'] < 0) * 1
df.head()

,user,age,f,m,sex-nan,Afghanistan,Albania,Algeria,American Samoa,Andorra,...,"Virgin Islands, British","Virgin Islands, U.s.",Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,country-nan,age-nan,age-negative
0,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5909125332c108365a26ccf0ee62636eee08215c,29,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,30,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,63268cce0d68127729890c1691f62d5be5abd87c,21,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,02871cd952d607ba69b64e2e107773012c708113,24,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
#artists = set(train['artist'])
#for artist in artists:
#    df[artist] = np.nan
#    for row in train.iterrows():
#        if row[1]['artist'] == artist:
#            df.loc[row[0],artist]=row[1]['plays']

In [255]:
#train_set_unstacked.head()['plays']
df.head()

,user,age,f,m,sex-nan,Afghanistan,Albania,Algeria,American Samoa,Andorra,...,"Virgin Islands, British","Virgin Islands, U.s.",Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,country-nan,age-nan,age-negative
0,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5909125332c108365a26ccf0ee62636eee08215c,29,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,30,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,63268cce0d68127729890c1691f62d5be5abd87c,21,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,02871cd952d607ba69b64e2e107773012c708113,24,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [252]:
#artists = set(train['artist'])
#for artist in artists:
#    df[artist] = np.nan
merged_data = df.merge(train_set_unstacked['plays'])
merged_data.head()

MergeError: No common columns to perform merge on

In [16]:
merged_data_test = df.merge(test)
merged_data_test.head()

,user,age,f,m,sex-nan,Afghanistan,Albania,Algeria,American Samoa,Andorra,...,"Virgin Islands, British","Virgin Islands, U.s.",Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,country-nan,Id,artist
0,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,42832,144ef525-85e9-40c3-8335-02c32d0861f3
1,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,324382,fbb375f9-48bb-4635-824e-4120273b3ba7
2,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,499045,d43d12a1-2dc9-4257-a2fd-0a3bb1081b86
3,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,706071,ea4dfa26-f633-4da6-a52a-f49ea4897b58
4,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,737504,cc7d4686-ea02-45fd-956e-94c1a322558c


In [9]:
user_dict = {}
for i in train['user']:
    user_dict[i] = {}
    
for i in train['user']:
    x = train.loc[train['user'] == i]
    for j in x:
        user_dict[i] = dict(zip(x.artist, x.plays))
        
testdf = pd.DataFrame.from_dict(user_dict, orient='index')
where_are_NaNs = np.isnan(testdf)
testdf[where_are_NaNs] = 0

In [8]:
X_train = merged_data[merged_data.columns[:246]]
Y_train = merged_data[merged_data.columns[246]]
X_test = merged_data[merged_data.columns[:246]]

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(X_train, Y_train)

In [18]:
# Merge Training Data with profiles_data Data
merged = profiles_data.merge(train)

In [182]:
# Dealing with Missing Data:  Naive Approach, complete cases
train = pd.read_csv('train.csv')
profiles_new = profiles_data.dropna()

merged_data = profiles_new.merge(train)
#merged_data1 = profiles_data.merge(train)

# Turn merged data into X_train and Y_train
X_train = merged_data[merged_data.columns[:5]]
Y_train = merged_data[merged_data.columns[5]]

# Encode artists using preprossing
le = preprocessing.LabelEncoder()
le.fit(X_train['artist'])
X_train['artist_code'] = le.transform(X_train['artist'])

le.fit(X_train['country'])
X_train['country_code'] = le.transform(X_train['country'])

le.fit(X_train['sex'])
X_train['sex'] = le.transform(X_train['sex'])

/Users/Dana/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Dana/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Dana/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydat

In [183]:
X_train = X_train.drop('artist', 1)
X_train = X_train.drop('country', 1)
X_train = X_train.drop('sex', 1)

In [17]:
rf = RandomForestClassifier(n_estimators=1,min_samples_split=5,random_state=4)
rf.fit(X_train, Y_train)

In [81]:
filename1 = 'X_train.p'
pickle.dump(df, io.open(filename1, 'wb'))

In [38]:
sum(merged_data['age-negative'])

697

In [50]:
X_train = merged_data[merged_data.columns[:247]]
X_train.head()

,user,age,f,m,sex-nan,Afghanistan,Albania,Algeria,American Samoa,Andorra,...,"Virgin Islands, British","Virgin Islands, U.s.",Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,country-nan,age-nan,age-negative
0,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,25,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### K-Means

Cluster the users into K classes

In [ ]:
from sklearn.cluster import KMeans

K = 12 # Number of classes
kmeans = KMeans(K) 
kmeans.fit(X_train)

In [ ]:
X_train['labels'] = kmeans.labels__

###User-Item Collaborative Systems

In [257]:
from sklearn.cross_validation import KFold
kf = KFold(n=len(train), n_folds=5, shuffle=False, random_state=3)

In [265]:
kfold_predictions = []
MAE = []

for train_index, test_index in kf:
    
    X_train, X_test = train.ix[train_index, :], train.ix[test_index, :]
    
    overall_mean = X_train['plays'].mean()
    overall_median = X_train['plays'].median()

    subdf = X_train[['artist', 'plays']]
    #artist_means = subdf.groupby('artist').mean()
    #artist_means['avgplays'] = artist_means['plays'] - overall_mean
    #del artist_means['plays']
    #artist_means = pd.DataFrame.to_dict(artist_means)['avgplays']
    artist_medians = subdf.groupby('artist').median()
    artist_medians['medplays'] = artist_medians['plays'] - overall_median
    del artist_medians['plays']
    artist_medians = pd.DataFrame.to_dict(artist_medians)['medplays']

    subdf = X_train[['user', 'plays']]
    #user_means = subdf.groupby('user').mean()
    #user_means['avgplays'] = user_means['plays'] - overall_mean
    #del user_means['plays']
    #user_means = pd.DataFrame.to_dict(user_means)['avgplays']
    user_medians = subdf.groupby('user').median()
    user_medians['medplays'] = user_medians['plays'] - overall_median
    del user_medians['plays']
    user_medians = pd.DataFrame.to_dict(user_medians)['medplays']
    
    predictions = []
    for i, j in zip(X_test['user'], X_test['artist']):
        '''
        try:
            p = overall_mean + user_means[i] + artist_means[j]
            #if p < 0:
            #    p = 0
            predictions.append(p)
        except:
            p = overall_mean + 0 + artist_means[j]
            #if p < 0:
            #    p = 0
            predictions.append(p)
        '''
        try:
            p = overall_median + user_medians[i] + artist_medians[j]
            if p < 0:
                p = 0
            predictions.append(p)
        except:
            p = overall_median + 0 + artist_medians[j]
            if p < 0:
                p = 0
            predictions.append(p)
    shift = min(predictions)
    #predictions = predictions + shift
    
    MAE.append(sum(abs(X_test['plays'] - predictions))/len(X_test))
    kfold_predictions.append(predictions)

In [212]:
MAE # MEAN: for shifting every prediciton up to min

[257.28789827309532,
 258.15533086042217,
 256.69504440356701,
 255.94572232779623,
 257.43756353338642]

In [210]:
MAE # MEAN: for setting = 0

[176.25186801713923,
 174.94202683803979,
 175.06217330929474,
 175.41404116109535,
 175.36080979232705]

In [260]:
MAE # MEDIAN: for shifting every prediciton up to min

[186.90224450967014,
 184.03921543851035,
 184.39845913346113,
 184.14110168828623,
 184.26433943872149]

In [266]:
MAE # MEDIAN: for setting = 0

[143.100447554097,
 141.88868611162258,
 142.14390928094099,
 141.43150280217725,
 141.47249566766149]

In [216]:
overall_mean = train['plays'].mean()

subdf = train[['artist', 'plays']]
artist_means = subdf.groupby('artist').mean()
artist_means['avgplays'] = artist_means['plays'] - overall_mean
del artist_means['plays']
artist_means = pd.DataFrame.to_dict(artist_means)['avgplays']

subdf = train[['user', 'plays']]
user_means = subdf.groupby('user').mean()
user_means['avgplays'] = user_means['plays'] - overall_mean
del user_means['plays']
user_means = pd.DataFrame.to_dict(user_means)['avgplays']

predictions = []
for i, j in zip(test['user'], test['artist']):
    p = overall_mean + user_means[i] + artist_means[j]
    if p < 0:
        p = 0
    predictions.append(p)

In [223]:
# Write out test solutions - MEAN
soln_file  = 'mean_adjust.csv'
test_file  = 'test.csv'

with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            p = predictions[int(id)-1]

            soln_csv.writerow([id, p])
# KAGGLE SCORE: 174.23682

In [267]:
# Write out test solutions - MEDIAN

overall_median = train['plays'].median()

subdf = train[['artist', 'plays']]
artist_medians = subdf.groupby('artist').median()
artist_medians['medplays'] = artist_medians['plays'] - overall_median
del artist_medians['plays']
artist_medians = pd.DataFrame.to_dict(artist_medians)['medplays']

subdf = train[['user', 'plays']]
user_medians = subdf.groupby('user').median()
user_medians['medplays'] = user_medians['plays'] - overall_median
del user_medians['plays']
user_medians = pd.DataFrame.to_dict(user_medians)['medplays']

predictions = []
for i, j in zip(test['user'], test['artist']):
    p = overall_median + user_medians[i] + artist_medians[j]
    if p < 0:
        p = 0
    predictions.append(p)

In [269]:
# Write out test solutions - MEDIAN
soln_file  = 'median_adjust.csv'
test_file  = 'test.csv'

with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            p = predictions[int(id)-1]

            soln_csv.writerow([id, p])
# KAGGLE SCORE: 141.24741